# 로지스틱 회귀 
이름은 회귀이지만 분류 모델임<br>
선형 회귀와 마찬가지로 선형 방정식을 학습함

*z = a(weight) + b(length) + c(diagonal) + d(height) + e(width) + f*<br>

a,b,c,d,e는 가중치 혹은 계수이고, z는 어느 값이든 될 수 있다.<br>
z가 아주 큰 값일 때 1, 아주 작은 값일 때 0을 주도록 하는 방법이 없을까? --> 시그모이드 함수(로지스틱 함수)



In [1]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [3]:
pd.unique(fish['Species'])

array(['Bream', 'Roach', 'Whitefish', 'Parkki', 'Perch', 'Pike', 'Smelt'],
      dtype=object)

In [2]:
fish_input = fish[['Weight','Length','Diagonal','Height','Width']]
fish_target = fish['Species'].to_numpy()

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state = 42)

# 표준화 전처리
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

# 도미와 빙어로 로지스틱 회귀 이진 분류 수행

In [6]:
# 도미와 빙어일 경우 True, 그 외 모두 False
bream_smelt_indexes = (train_target=='Bream') | (train_target=='Smelt')

# 도미와 빙어 데이터만 따로 골라내기
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

In [12]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

LogisticRegression()

In [13]:
print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [15]:
# 예측값을 확률로 반환
# 첫번째 열이 음성클래스, 두번째 열이 양성클래스
# 사이킷런은 타깃값을 알파벳순으로 정렬. 따라서 bream, smelt중 bream이 음성클래스
print(lr.predict_proba(train_bream_smelt[:5]))

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


In [17]:
# 로지스틱 회귀 계수 확인
print(lr.coef_, lr.intercept_)

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


In [23]:
from scipy.special import expit
# decision_function클래스로 z값 출력 가능
decision = lr.decision_function(train_bream_smelt[:5])
print(decision)

# 위에서 얻은 z값을 시그모이드 함수에 통과시키면 확률 get
# 양성 클래스에 대한 z값 반환
print(expit(decision))


[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]
[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


# 로지스틱 회귀로 다중 분류 수행
로지스틱 회귀는 기본적으로 반복적인 알고리즘을 사용합니다.

max_iter 매개변수에서 반복 횟수 지정. 디폴트는 100

로지스틱 회귀는 릿지 회귀와 같이 계수의 제곱을 규제함. 이를 L2규재라고도 함.

릿지 회귀에서 alpha 매개변수로 규제의 양을 정했음. 로지스틱에서 규제의 매개변수는 C( C는 alpha와 반대로 작을수록 규제가 커짐)

In [37]:
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))
# 과대적합이나 과소적합은 없음. Good

0.9327731092436975
0.925


In [38]:
lr.predict(test_scaled[:5])

array(['Perch', 'Smelt', 'Pike', 'Roach', 'Perch'], dtype=object)

In [44]:
proba = lr.predict_proba(test_scaled[:5])

# 예측 샘플 5개로 5개의 행
# 물고기 7마리 분류로 7개의 열
print(np.round(proba, decimals=3))
print(lr.classes_)

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [47]:
# 5개의 특성을 사용하므로 열은 5개
# 근데 행이 7개 >> 이진분류에서 보았던 z를 7번 계산한다는 뜻
print(lr.coef_.shape, lr.intercept_.shape)

# 이진분류에서는 시그모이드 함수를 사용했다면 다중분류는 이와달리 소프트맥스 함수를 사용

(7, 5) (7,)


In [50]:
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]


In [53]:
from scipy.special import softmax
proba = softmax(decision,axis=1)
print(np.round(proba, decimals=3))
print(test_target[:5])

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
['Perch' 'Smelt' 'Pike' 'Whitefish' 'Perch']


# 로지스틱 회귀 정리
## 이진 분류
하나의 선형 방정식을 훈련<br>
해당 방정식의 출력값을 시그모이드 함수에 통과시켜 0~1 사이의 값을 만듦<br>
이 값이 양성 클래스에 대한 확률<br>
음성 클래스 확률은 1애서 양성 클래스 값 뺴면 됨.

---
## 다중 분류
클래스 개수(물고기 개수)만큼 방정식을 훈련<br>
각 방정식의 출력값을 소프트맥스 함수에 통과시켜 전체 클래스에 대한 합이 항상 1이 되도록 만듦<br>
이 값을 각 클래스에 대한 확률로 이해할 수 있음.